# Main code for analysis of a dataset

#### includes dashboard, evaluation, parameter k tuning, and validation with labeled change point conbined datasets

In [341]:
import pandas as pd
import numpy as np
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
from scipy.fft import rfft, rfftfreq, rfft, irfft, fftfreq, fft
import scipy
import math
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter1d
import plotly.express as px
import plotly.subplots as ps
import dash_core_components as dcc
# from datetime import datetime, timedelta
import datetime
import pickle
import random
import dataframe_image as dfi

##### merging datasets

In [198]:
def merge_datasets(path1, name1, index1, path2, name2, index2):
    """load two specified datasets and paste the second directly after the first
    after for each subtracting the mean and dividing by the standard deviation"""
    
    df1 = pd.read_excel(path1)[index1[0]:index1[1]].reset_index()
    df2 = pd.read_excel(path2)[index2[0]:index2[1]].reset_index()
    
    sig1 = df1.Temp
    sig2 = df2.Temp
    N=12*24
    win_day = scipy.signal.windows.tukey(M=N, alpha=0.5, sym=False)
    df1['avg'] = scipy.signal.convolve(sig1, win_day, mode='same') / sum(win_day)
    df2['avg'] = scipy.signal.convolve(sig2, win_day, mode='same') / sum(win_day)
    df1['std'] = df1['Temp'].rolling(N, min_periods=1).std()
    df1['std'][0] = 1
    df2['std'] = df2['Temp'].rolling(N, min_periods=1).std()
    df2['std'][0] = 1
    
    df1['temp-avg'] = df1['Temp']-df1['avg']
    df2['temp-avg'] = df2['Temp']-df2['avg']
    
    df1['temp-avg/std'] = df1['temp-avg']/df1['std']
    df2['temp-avg/std'] = df2['temp-avg']/df2['std']

    df1 = df1.iloc[1000:-1000]
    df2 = df2.iloc[1000:-1000]
    
    df3 = pd.concat([df1, df2], ignore_index=True)
    df4 = df3[['EventDt', 'temp-avg/std']].rename(columns = {'temp-avg/std':'Temp'})
    
    df4['EventDt'] = pd.date_range(str(df4['EventDt'][0]), periods=len(df4), freq='5T')
    
    df4['Switch'] = 0
    df4['Switch'][len(df1)-1] = 1
    
    df4.to_excel('01 - Raw Data/combinations/random combinations/data_{}_{}.xlsx'.format(name1, name2))

##### preprocessing

In [4]:
def convolutions(data):
    """compute convolutions of the data on different timescales
    based on a Tukey window convolutions of lengths of a week/day/hour"""
    
    sig = data.Temp
    win_week = scipy.signal.windows.tukey(M=12*24*7, alpha=0.5, sym=False)
    data['gauss_1week'] = scipy.signal.convolve(sig, win_week, mode='same') / sum(win_week)
    win_day = scipy.signal.windows.tukey(M=12*24, alpha=0.5, sym=False)
    data['gauss_1day'] = scipy.signal.convolve(sig, win_day, mode='same') / sum(win_day)
    win_hour = scipy.signal.windows.tukey(M=12, alpha=0.5, sym=False)
    data['gauss_1hour'] = scipy.signal.convolve(sig, win_hour, mode='same') / sum(win_hour)
    
    return data

In [5]:
def minmaxstd(data, N):
    """compute the minimum and maximum temperatures, and standard deviation
    with a rolling window of specified length"""
    
    data['max Temp'] = data['Temp'].rolling(N, min_periods=1).max()
    data['min Temp']  = data['Temp'].rolling(N, min_periods=1).min()
    data['std Temp'] = data['Temp'].rolling(N, min_periods=1).std()
    data['std Temp'][0] = 0
    
    return data

In [6]:
def filter_outliers(data, lower, upper):
    """filter outliers based on a lower and upper percentile of the 1hour convoluted data"""
    
    data = data[data['gauss_1hour']<np.percentile(data['gauss_1hour'], upper)]
    data = data[data['gauss_1hour']>np.percentile(data['gauss_1hour'], lower)]
    
    return data

##### spectral analysis

In [7]:
def compute_spectrogram(df, N):
    """computes the spectrogram of the temperature signal"""
    
    f, t, Sxx = scipy.signal.spectrogram(df['Temp - avg'], fs=1/300,  nperseg=N)
    
    return f, t, Sxx

In [383]:
def octave_intensities(N, f, t, Sxx):
    """computes the absolute and relative octave intensities based on the spectrogram"""
    
    delta_t = 5*60
    T = N*delta_t
    delta_f = 1/T
    F = N*delta_f/2
    M = int(round(np.log(F/delta_f)))
    
    parts = M
    borders = [(delta_f*math.exp((i/M)*np.log(F/delta_f))) for i in range(M+1)]
    indi = [len(f[f<=borders[i]]) for i in range(len(borders))]
    metrics_relative = dict()
    metrics = dict()
    for d in range(len(t)):
        tindex = t[d]
        F = [Sxx[i][d] for i in range(len(Sxx))]
        metrics[t[d]] = dict()
        metrics_relative[t[d]] = dict()
        sumf = sum(F[:])
        for b in range(1, parts+1):
            metrics_relative[t[d]][b] = sum(F[indi[b-1]:indi[b]])/sumf
            metrics[t[d]][b] = sum(F[indi[b-1]:indi[b]])
            
    return metrics, metrics_relative

In [9]:
def entropy_spectrum(N, intensities):
    """computes the entropy/Kullback-Leibner divergence/information gain based on the octave intensities"""
    
    delta_t = 5*60
    T = N*delta_t 
    delta_f = 1/T
    F = N*delta_f/2
    M = int(round(np.log(F/delta_f)))
    
    Poct = intensities
    Qpinkyoct = 1/M
    KL = dict()
    for i in Poct:
        KL[i] = sum([Poct[i][j]/Qpinkyoct for j in range(1,M+1)])
    
    return KL

In [10]:
def PCA_intensities(intensities):
    """computes the first two principal components of the octave intensities"""
    
    metrics = intensities
    df_metrics = pd.DataFrame(metrics).T
    
    df = df_metrics.rename({1: 'octave 1', 
                            2: 'octave 2', 
                            3: 'octave 3',
                            4: 'octave 4',
                            5: 'octave 5'}, axis=1)
    features = df.columns

    pca = PCA(2)
    components = pca.fit_transform(df[features])
    labels = {str(i): f"PC {i+1} ({var:.1f}%)" for i, var in enumerate(pca.explained_variance_ratio_ * 100)}
    
    return components

##### Evaluation

In [369]:
def warning_additive(df, name, period, l, h, k):
    """computes warnings based on threshold values 
    computed with percentile values calculated over a specified period and a factor k in an additive formula
    """
    
    warnings = list()
    skip = period
    values = list()
    calculate = 1
    for value in df[name]:
        if skip !=0:
            warnings.append(np.nan)
            values.append(value)
            skip -= 1
            continue
        elif skip == 0:
            calculate -= 1
            if calculate == 0:
                l_percentile = np.percentile(values, l)
                h_percentile = np.percentile(values, h)
                median = np.median(values)
                low = median+k*(l_percentile-median)
                high = median+k*(h_percentile-median)
            if value > high:
                warnings.append(1)
                skip = period
                calculate = 1   
            elif value < low:
                warnings.append(-1)
                skip = period
                calculate = 1
            else:
                warnings.append(0)
                
    return warnings

In [368]:
def warning_multiplicative(df, name, period, l, h, k):
    """computes warnings based on threshold values 
    computed with percentile values calculated over a specified period and a factor k in a multiplicative formula
    """
    
    warnings = list()
    skip = period
    values = list()
    calculate = 1
    for value in df[name]:
        if skip !=0:
            warnings.append(np.nan)
            values.append(value)
            skip -= 1
            continue
        elif skip == 0:
            calculate -= 1
            if calculate == 0:
                l_percentile = np.percentile(values, l)
                h_percentile = np.percentile(values, h)
                median = np.median(values) # kan weg
                low = (l_percentile/median)/k*median
                high = (h_percentile/median)*k*median
            if value > high:
                warnings.append(1)
                skip = period
                calculate = 1
            elif value < low:
                warnings.append(-1)
                skip = period
                calculate = 1
            else:
                warnings.append(0)
                
    return warnings

In [370]:
def evaluation(df, dfmm, dfc, lines, dfKL, components, t_dates, l, h, k_list):
    """uses the additive and multiplicative warning functions to copmute warnings for all performance indicators"""
    
    df['warnings_temp'] = warning_additive(df=df, name='Temp', period=12*24*31, l=l, h=h, k=k_list[0])
    
    ts = ' Temp'
    for column in ['min', 'max']:
        dfmm['warnings_{}'.format(column)] = warning_additive(df=dfmm, name=column+ts, period=12*24*31, l=l, h=h, k=k_list[1])
    for column in ['std']:
        dfmm['warnings_{}'.format(column)] = warning_multiplicative(df=dfmm, name=column+ts, period=12*24*31, l=l, h=h, k=k_list[3])
    
    dfc['warnings_avg'] = warning_additive(df=dfc, name='gauss_1day', period=12*24*31, l=l, h=h, k=k_list[2])
    
    dfl = pd.DataFrame(lines)
    for column in dfl:
        dfl['warnings_oct{}'.format(column)] = warning_multiplicative(df=dfl, name=column, period=31, l=l, h=h, k=k_list[4])
        
    dfKL['warnings_IG'] = warning_multiplicative(df=dfKL, name=1, period=31, l=l, h=h, k=k_list[5])
    
    dfpca = pd.DataFrame(components)
    for column in dfpca:
        dfpca['warnings_PC{}'.format(column)] = warning_additive(df=dfpca, name=column, period=31, l=l, h=h, k=k_list[6])
        
    return df, dfmm, dfc, dfl, dfKL, dfpca

##### count warnings

In [14]:
def warnings_stats(dfT, dfMMS, dfAVG, dfO, dfIG, dfPC):
    """creates a dataframe with counts of all warnings for all performance indicators, 
    the average warnings per year for each performance indicator,
    and the average divided by the amount of variables the performance indicator consists of"""
    
    years = len(dfIG)/365
    k = pd.DataFrame(index=['Total warnings', 'Average per year', 'Average divided by amount of variables'],
                     columns=['Temperature', 'Min/Max temperature', 'Average temperature', 'Temperature SD',
                              'Octave intensities', 'Information gain', 'Principal components'])
    
    k['Temperature']['Total warnings'] = dfT['warnings_temp'].abs().sum()
    k['Temperature']['Average per year'] = k['Temperature']['Total warnings']/years
    k['Temperature']['Average divided by amount of variables'] = k['Temperature']['Average per year']
    
    k['Min/Max temperature']['Total warnings'] = dfMMS[['warnings_min', 'warnings_max']].abs().sum().sum()
    k['Min/Max temperature']['Average per year'] = k['Min/Max temperature']['Total warnings']/years
    k['Min/Max temperature']['Average divided by amount of variables'] = k['Min/Max temperature']['Average per year']/2
    
    k['Average temperature']['Total warnings'] = dfAVG['warnings_avg'].abs().sum()
    k['Average temperature']['Average per year'] = k['Average temperature']['Total warnings']/years
    k['Average temperature']['Average divided by amount of variables'] = k['Average temperature']['Average per year']
    
    k['Temperature SD']['Total warnings'] = dfMMS['warnings_std'].abs().sum()
    k['Temperature SD']['Average per year'] = k['Temperature SD']['Total warnings']/years
    k['Temperature SD']['Average divided by amount of variables'] = k['Temperature SD']['Average per year']
    
    k['Octave intensities']['Total warnings'] = dfO[['warnings_oct1', 'warnings_oct2', 'warnings_oct3', 'warnings_oct4', 'warnings_oct5']].abs().sum().sum()
    k['Octave intensities']['Average per year'] = k['Octave intensities']['Total warnings']/years
    k['Octave intensities']['Average divided by amount of variables'] = k['Octave intensities']['Average per year']/5
    
    k['Information gain']['Total warnings'] = dfIG['warnings_IG'].abs().sum()
    k['Information gain']['Average per year'] = k['Information gain']['Total warnings']/years
    k['Information gain']['Average divided by amount of variables'] = k['Information gain']['Average per year']
    
    k['Principal components']['Total warnings'] = dfPC[['warnings_PC0', 'warnings_PC1']].abs().sum().sum()
    k['Principal components']['Average per year'] = k['Principal components']['Total warnings']/years
    k['Principal components']['Average divided by amount of variables'] = k['Principal components']['Average per year']/2
    
    return k  

##### evaluation combined datasets with labeled changepoint

In [174]:
def days_warning(df, dfT, dfMMS, dfAVG, dfO, dfIG, dfPC, t_dates):
    """computes the amount of time within which a warning was raised for each performance indicator relative to a labeled changepoint
    to be used with the labeled changepoint combined datasets"""
    
    day0 = list(df['EventDt'][df['Switch'] == 1])[0]
    
    time_till_warning = dict()
    
    for index, row in dfT[dfT['EventDt']>=day0][['EventDt', 'warnings_temp']].fillna(0).iterrows():
        if abs(row['warnings_temp']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Temperature warning'] = time
            break
            
    for index, row in dfMMS[dfMMS['EventDt']>=day0][['EventDt', 'warnings_min', 'warnings_max']].fillna(0).iterrows():
        if abs(row['warnings_min']) == 1 or abs(row['warnings_max']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Temperature Min/Max warning'] = time
            break
            
    for index, row in dfAVG[dfAVG['EventDt']>=day0][['EventDt', 'warnings_avg']].fillna(0).iterrows():
        if abs(row['warnings_avg']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Temperature average warning'] = time
            break
            
    for index, row in dfMMS[dfMMS['EventDt']>=day0][['EventDt', 'warnings_std']].fillna(0).iterrows():
        if abs(row['warnings_std']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Temperature SD warning'] = time
            break
            
    dfO['EventDt'] = t_dates
    for index, row in dfO[dfO['EventDt']>=day0][['EventDt', 'warnings_oct1', 'warnings_oct2', 'warnings_oct3', 'warnings_oct4', 'warnings_oct5']].fillna(0).iterrows():
        if abs(row['warnings_oct1']) == 1 or abs(row['warnings_oct2']) == 1 or abs(row['warnings_oct3']) == 1 or abs(row['warnings_oct4']) == 1 or abs(row['warnings_oct5']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Octave intensities warning'] = time
            break
            
    dfIG['EventDt'] = t_dates
    for index, row in dfIG[dfIG['EventDt']>=day0][['EventDt', 'warnings_IG']].fillna(0).iterrows():
        if abs(row['warnings_IG']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Information gain warning'] = time
            break
            
    dfPC['EventDt'] = t_dates
    for index, row in dfPC[dfPC['EventDt']>=day0][['EventDt', 'warnings_PC0', 'warnings_PC1']].fillna(0).iterrows():
        if abs(row['warnings_PC0']) == 1 or abs(row['warnings_PC1']) == 1:
            time = row['EventDt'] - day0
            time_till_warning['Principal components warning'] = time
            break
            
    return time_till_warning

##### analysis

In [377]:
def analyze_data(path, name, N, lower=0, upper=100, index=None, k_list=[1.125 for i in range(7)], save='you forgot to name'): 
    """loads a dataset and performs the full analysis using the above specified functions
    plots all computed performance indicators and warnings in a dashboard"""
    
    #load data from path and preprocess
    df = pd.read_excel(path)
    if index:
        df = df[index[0]:index[1]]
        
    dfmm = minmaxstd(df, N)    
    dfc = convolutions(dfmm)
    dfc['Temp - avg'] = dfc['Temp'] - dfc['gauss_1week']
    dfcf = filter_outliers(dfc, lower, upper)
    
    #
    ###
    #####
    #PLOT BASIC PERFORMANCE INDICATORS
    fig, axs = plt.subplots(15, figsize=(10,20), constrained_layout=True)
    fig.suptitle('Analysis Dataset {}'.format(name), y=1.01)
    
    #plot original temperature data
    axs[0].plot(df['EventDt'], df['Temp'], label='Temperature')
    axs[0].set_title("Temperature")
    axs[0].set_xlabel("Date")
    axs[0].set_ylabel("Temperature [°C]")
    axs[0].legend(['Temperature'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    
    #plot daily minimum and maximum temperature
    axs[2].plot(dfmm['EventDt'], dfmm['max Temp'], label='max Temperature', color='red')
    axs[2].plot(dfmm['EventDt'], dfmm['min Temp'], label='min Temperature', color= 'royalblue')
    axs[2].set_title("Daily minimum and maximum temperature")
    axs[2].set_xlabel("Date")
    axs[2].set_ylabel("Temperature [°C]")
    axs[2].legend(['Maximum temperature', 'Minimum temperature'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    
    #plot one day Tukey window convoluted temperature as average temperature
    axs[4].plot(dfc['EventDt'], dfc['gauss_1day'], label='avg Temperature')
    axs[4].set_title("Temperature average (1-day Tukey convolution)")
    axs[4].set_xlabel("Date")
    axs[4].set_ylabel("Temperature [°C]")
    axs[4].legend(['Average temperature'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    
    #plot daily standard deviation of the temperature
    axs[6].plot(dfmm['EventDt'], dfmm['std Temp'], label='SD')
    axs[6].set_title("Daily temperature standard deviation")
    axs[6].set_xlabel("Date")
    axs[6].set_ylabel("Temperature [°C]")
    axs[6].legend(['Temperature SD'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    #####
    ###
    #
    
    #SPECTRAL ANALYSIS
    f, t, Sxx = compute_spectrogram(dfcf, N)
    
    #create corresponding date xaxis for the spectral performance indicators
    end_date = dfcf['EventDt'].iloc[-1]
    start_date = dfcf['EventDt'].iloc[0]
    timestep = (end_date - (start_date + datetime.timedelta(seconds=t[0])))/len(t)
    xmin, xmax = axs[0].get_xlim()
    t_dates = [start_date + timestep*i for i in range(1, len(t)+1)]
    
    metrics, rel = octave_intensities(N, f, t, Sxx)
    lines = dict()
    for i in range(1, 5+1):
        line = list()
        for m in metrics:
            line.append(metrics[m][i])
        lines[i] = line
    KL = entropy_spectrum(N, metrics)
    dfKL = pd.DataFrame(zip(*KL.items())).T
    
    #PCA
    components = PCA_intensities(metrics)
    
    #
    ###
    #####
    #PLOT SPECTRAL PERFORMANCE INDICATORS
    #plot the spectrogram
    axs[8].pcolormesh(t_dates, f, np.log10(Sxx), shading='gouraud')
    axs[8].set_xlim((xmin,xmax))
    axs[8].set_yscale('symlog')
    axs[8].set_title("Spectrogram")
    axs[8].set_xlabel("Date")
    axs[8].set_ylabel("Frequency [Hz]")
    sm = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(vmin=0, vmax=1))
    cbar = fig.colorbar(sm, ax=axs[8], location='right', ticks=[0,1], aspect=10)
    cbar.ax.set_yticklabels(['Low', 'High'])
    
    #plot the absolute octave intensities as a heatmap line per octave
    for i in range(1, len(lines)+1):
        y = [i for j in range(len(lines[i]))]
        axs[9].scatter(x=t_dates,y=y, c=cm.viridis_r(np.abs([k/max(lines[i]) if k!=0 else 0 for k in lines[i]])), edgecolor='none')
    axs[9].set_yticks([1, 2, 3, 4, 5])
    axs[9].set_title("Sum of intensities per octave")
    axs[9].set_xlabel("Date")
    axs[9].set_ylabel("Octave")
    sm = plt.cm.ScalarMappable(cmap=cm.viridis_r, norm=plt.Normalize(vmin=0, vmax=1))
    cbar = fig.colorbar(sm, ax=axs[9], location='right', ticks=[0,1], aspect=10)
    cbar.ax.set_yticklabels(['Low', 'High'])
    
    #plot entropy = information gain of the spectrum
    axs[11].plot(t_dates, dfKL[1], label='KL divergence', color='g')
    axs[11].set_title("Information gain")
    axs[11].set_xlabel("Date")
    axs[11].set_ylabel("Kullback-Leibner divergence")
    axs[11].legend(['Kullback-Leibner divergence'], loc='center left', bbox_to_anchor=(1, 0.5),
                   fancybox=True, shadow=True)
    
    #plot first two principal components of the ocatve intensities
    axs[13].set_prop_cycle('color',['limegreen', 'gold'])
    axs[13].plot(t_dates, components)
    axs[13].set_title("Principal components of octave intensities")
    axs[13].set_xlabel("Date")
    axs[13].set_ylabel("Principal component")
    axs[13].legend(['PC 1', 'PC 2'], loc='center left', bbox_to_anchor=(1, 0.5),
                   fancybox=True, shadow=True)
    #####
    ###
    #
    
    #EVALUATION (warnings)
    dfT, dfMMS, dfAVG, dfO, dfIG, dfPC = evaluation(df, dfmm, dfc, lines, dfKL, components, t_dates, l=1, h=99, k_list=k_list)
    
    #
    ###
    #####
    #PLOT WARNINGS (EVALUATION)
    #plot temperature warnings
    axs[1].plot(dfT['EventDt'], dfT['warnings_temp'])
    axs[1].set_title("Temperature evaluation")
    axs[1].set_ylim([-1,1])
    axs[1].axes.get_xaxis().set_visible(False)
    axs[1].legend(['Temperature warnings'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    axs[0].get_shared_x_axes().join(axs[0], axs[1])
    
    #plot minimum and maximum warnings
    axs[3].set_prop_cycle('color',['royalblue', 'red'])
    axs[3].plot(dfMMS['EventDt'], dfMMS[['warnings_min', 'warnings_max']])
    axs[3].set_title("Temperature minimum and maximum evaluation")
    axs[3].set_ylim([-1,1])
    axs[3].axes.get_xaxis().set_visible(False)
    axs[3].legend(['Minimum temperature warnings', 'Maximum temperature warnings'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    axs[2].get_shared_x_axes().join(axs[2], axs[3])
    
    #plot average temperature warnings
    axs[5].plot(dfAVG['EventDt'], dfAVG['warnings_avg'])
    axs[5].set_title("Temperature average evaluation")
    axs[5].set_ylim([-1,1])
    axs[5].axes.get_xaxis().set_visible(False)
    axs[5].legend(['Average temperature warnings'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    axs[4].get_shared_x_axes().join(axs[4], axs[5])
    
    #plot temperature standard deviation warnings
    axs[7].plot(dfMMS['EventDt'], dfMMS['warnings_std'])
    axs[7].set_title("Temperature standard deviation evaluation")
    axs[7].set_ylim([-1,1])
    axs[7].axes.get_xaxis().set_visible(False)
    axs[7].legend(['Temperature SD warnings'], loc='center left', bbox_to_anchor=(1, 0.5),
                  fancybox=True, shadow=True)
    axs[6].get_shared_x_axes().join(axs[6], axs[7])
    
    #plot octave intensities warnings
    axs[10].plot(t_dates, dfO[['warnings_oct1', 'warnings_oct2', 'warnings_oct3', 'warnings_oct4', 'warnings_oct5']]) 
    axs[10].set_title("Octave intensities evaluation")
    axs[10].set_ylim([-1,1])
    axs[10].axes.get_xaxis().set_visible(False)
    axs[10].legend(['warnings oct1', 'warnings oct2', 'warnings oct3', 'warnings oct4', 'warnings oct5'], loc='center left', bbox_to_anchor=(1, 0.5),
                   fancybox=True, shadow=True)
    axs[9].get_shared_x_axes().join(axs[9], axs[10])
    
    #plot information gain warnings
    axs[12].plot(t_dates, dfIG['warnings_IG'], color='g')
    axs[12].set_title("Information gain evaluation")
    axs[12].set_ylim([-1,1])
    axs[12].axes.get_xaxis().set_visible(False)
    axs[12].legend(['Information gain warnings'], loc='center left', bbox_to_anchor=(1, 0.5),
                   fancybox=True, shadow=True)
    axs[11].get_shared_x_axes().join(axs[11], axs[12])
    
    #plot principal components warnings
    axs[14].set_prop_cycle('color',['limegreen', 'gold'])
    axs[14].plot(t_dates, dfPC[['warnings_PC0', 'warnings_PC1']])
    axs[14].set_title("Principal components of octave intensities evaluation")
    axs[14].set_ylim([-1,1])
    axs[14].axes.get_xaxis().set_visible(False)
    axs[14].legend(['warnings PC1', 'warnings PC2'], loc='center left', bbox_to_anchor=(1, 0.5),
                   fancybox=True, shadow=True)
    axs[13].get_shared_x_axes().join(axs[13], axs[14])
    
    #save figure to specified path
    fig.savefig('thesis plots/{}.png'.format(save), bbox_inches='tight')
    #####
    ###
    #

    #COUNT WARNINGS
#     df_warnings = warnings_stats(dfT, dfMMS, dfAVG, dfO, dfIG, dfPC)
#     return name, k, df_warnings

    #EVALUATION LABELED CHANGEPOINT DATASETS
#     time_till_warning = days_warning(df, dfT, dfMMS, dfAVG, dfO, dfIG, dfPC, t_dates)
#     return time_till_warning

##### example code: run to get dashboard

In [ ]:
analyze_data('01 - Raw Data/data_G.xlsx', 'G', lower=5, upper=70, N=(12*24), k=2)

##### using "COUNT WARNINGS" to get a table of warnigns per performance indicator per value of k

In [ ]:
warnings_dict_k125 = dict()
for i in [x for x in range(1,21) if x not in [2, 12, 13]]:
    for k in [(9/8)**n for n in range(1,11)]:
        path = '01 - Raw Data/Delivery 2 (enkel koelkast)/data_{}.xlsx'.format(i)
        warnings_dict_k125['warnings_ds{}_k={}'.format(i, round(k, 3))] = analyze_data(path, name='{}'.format(i), lower=5, upper=70, N=(12*24), k=k)       

In [357]:
k_dict = dict()
for k in [round((9/8)**n, 2) for n in range(1,11)]:
    k_dict['k = {}'.format(k)] = dict()
    tables_k = [warnings_dict_k125['warnings_ds{}_k={}'.format(i, k)][2] for i in [x for x in range(1,21) if x not in [2, 12, 13]]]
    l1 = list()
    l2 = list()
    l3 = list()
    l4 = list()
    l5 = list()
    l6 = list()
    l7 = list()
    for t in tables_k:
        l1.append(t['Temperature'].loc['Average per year'])
        l2.append(t['Min/Max temperature'].loc['Average per year'])
        l3.append(t['Average temperature'].loc['Average per year'])
        l4.append(t['Temperature SD'].loc['Average per year'])
        l5.append(t['Octave intensities'].loc['Average per year'])
        l6.append(t['Information gain'].loc['Average per year'])
        l7.append(t['Principal components'].loc['Average per year'])   
    k_dict['k = {}'.format(k)]['Temperature'] = sum(l1)/len(l1)
    k_dict['k = {}'.format(k)]['Min/Max temperature'] = sum(l2)/len(l2)
    k_dict['k = {}'.format(k)]['Average temperature'] = sum(l3)/len(l3)
    k_dict['k = {}'.format(k)]['Temperature SD'] = sum(l4)/len(l4)
    k_dict['k = {}'.format(k)]['Octave intensities'] = sum(l5)/len(l5)
    k_dict['k = {}'.format(k)]['Information gain'] = sum(l6)/len(l6)
    k_dict['k = {}'.format(k)]['Principal components'] = sum(l7)/len(l7)

In [358]:
for k in [round((9/8)**n, 2) for n in range(1,11)]:
    tables_k = [warnings_dict_k125['warnings_ds{}_k={}'.format(i, k)][2] for i in [x for x in range(1,21) if x not in [2, 12, 13]]]
    avgw_list = list() 
    for t in tables_k:
        avg_warnings = t[['Temperature', 'Min/Max temperature', 'Average temperature', 'Temperature SD',
           'Octave intensities', 'Information gain', 'Principal components']].loc['Average per year'].sum()/ 7
        avgw_list.append(avg_warnings)
    k_dict['k = {}'.format(k)]['Total average'] = sum(avgw_list)/len(avgw_list)

In [359]:
k_dataframe = pd.DataFrame(k_dict).round(1)

In [360]:
k_dataframe

,k = 1.12,k = 1.27,k = 1.42,k = 1.6,k = 1.8,k = 2.03,k = 2.28,k = 2.57,k = 2.89,k = 3.25
Temperature,7.7,6.2,4.7,3.7,3.1,2.4,2.1,1.8,1.5,1.2
Min/Max temperature,3.6,2.3,1.8,1.4,1.2,1.0,0.9,0.7,0.6,0.5
Average temperature,2.4,2.0,1.5,1.0,1.0,0.7,0.8,0.7,0.6,0.5
Temperature SD,1.5,1.0,0.8,0.6,0.4,0.3,0.3,0.2,0.1,0.1
Octave intensities,14.5,12.6,11.1,9.5,8.3,7.1,6.1,5.5,5.1,4.5
Information gain,2.0,1.6,1.3,1.1,1.0,0.8,0.7,0.6,0.5,0.4
Principal components,3.3,2.3,1.7,1.3,1.1,1.0,0.9,0.7,0.6,0.6
Total average,5.0,4.0,3.3,2.7,2.3,1.9,1.7,1.5,1.3,1.1


In [361]:
dfi.export(k_dataframe, 'k dataframe.png', max_cols=-1)

In [38]:
# with open('paramk125dict.pkl', 'wb') as f:
#     pickle.dump(warnings_dict_k125, f)
        
# with open('paramk125dict.pkl', 'rb') as f:
#     dictie = pickle.load(f)

##### using "EVALUATION LABELED CHANGEPOINT DATASETS" to get a table with the number of days within which a warning was raised relative to the labeled changepoint for the combined datasets

In [203]:
candidates = [3,4,5,6,7,8,9,10,11,14,16,17,18,19]
candidate_combinations = [(i, j) for i in candidates for j in candidates if i != j]
chosen_combinations = random.choices(candidate_combinations, k=10)

In [ ]:
for i in chosen_combinations:
    merge_datasets(path1='01 - Raw Data/Delivery 2 (enkel koelkast)/data_{}.xlsx'.format(i[0]), name1=i[0], index1=(-12*24*180-2000, -1), 
                   path2='01 - Raw Data/Delivery 2 (enkel koelkast)/data_{}.xlsx'.format(i[1]), name2=i[1], index2=(0, 12*24*180+2000))

In [ ]:
times_till_warning = dict()
for i in chosen_combinations:
    path = '01 - Raw Data/combinations/random combinations/data_{}_{}.xlsx'.format(i[0], i[1])
    k_list = [2, 1.2, 1.125, 1.125, 2.3, 1.125, 1.2]
    times_till_warning['{}, {}'.format(i[0], i[1])] = analyze_data(path, '{}, {}'.format(i[0], i[1]), lower=0, upper=100, N=(12*24), k_list=k_list, save='validation/dash val {}, {}'.format(i[0], i[1]))

In [380]:
for times in times_till_warning:
    for time in times_till_warning[times]:
        times_till_warning[times][time] = int((times_till_warning[times][time].round(freq='D')+timedelta(days=1)).days)

In [381]:
times = pd.DataFrame(times_till_warning).fillna(999).astype(int)#.replace(404, '')
times_highlighted = times.style.highlight_min(color = 'lightgreen', axis = 0)
times_highlighted

,"6, 3","10, 5","8, 5","17, 9","14, 9","9, 18","17, 16","6, 5","3, 16","8, 19"
Octave intensities warning,1,1,2,1,1,1,1,2,2,2
Principal components warning,1,1,1,9,2,43,2,2,1,1
Temperature warning,999,4,31,9,9,102,3,999,1,60
Temperature Min/Max warning,999,4,31,9,34,3,33,999,3,45
Temperature average warning,999,31,13,134,108,32,87,999,87,53
Temperature SD warning,999,4,16,135,34,45,88,999,88,44
Information gain warning,999,5,113,9,135,45,88,999,88,44


In [382]:
dfi.export(times_highlighted, 'warning times highlight.png', max_cols=-1)